Preprocessing Antagonistic Drug Interactions from DrugCombDB

In [1]:
import pandas as pd

In [ ]:
df_drug_comb = pd.read_excel(r"...path_to\RDsqr-KG\Datasets\DrugCombDB\SynDrugComb_external.xlsx", sheet_name='Drugbank_antagonism')
df_drug_map = pd.read_csv(r"...path_to\RDsqr-KG\Datasets\Vocabulary_links\drug links.csv")
df_drug_status = pd.read_csv(r"...path_to\RDsqr-KG\Preprocessed_datasets\drug_status.csv")

In [3]:
df_drug_comb.head()

,Interaction_A(DrugBank_ID),Interaction_B(DrugBank_ID)
0,DB01183,DB06800
1,DB01250,DB01109
2,DB01586,DB01054
3,DB01212,DB01032
4,DB01326,DB01032


In [4]:
df_drug_comb = df_drug_comb[df_drug_comb['Interaction_A(DrugBank_ID)'].str.startswith('DB', na=False)]

In [5]:
df_drug_comb.isna().sum()

Interaction_A(DrugBank_ID)    0
Interaction_B(DrugBank_ID)    0
dtype: int64

In [6]:
df_drug_comb.dropna(inplace=True)

In [7]:
df_drug_comb.isna().sum()

Interaction_A(DrugBank_ID)    0
Interaction_B(DrugBank_ID)    0
dtype: int64

In [8]:
df_drug_comb.drop_duplicates(inplace=True)

In [9]:
df_drug_comb.shape

(13397, 2)

In [10]:
df_nddi = pd.merge(df_drug_comb, df_drug_map, left_on='Interaction_A(DrugBank_ID)', right_on='DrugBank ID', how='left')

In [11]:
df_nddi = pd.merge(df_nddi, df_drug_map[['DrugBank ID', 'Name']], left_on='Interaction_B(DrugBank_ID)', right_on='DrugBank ID', how='left')

In [12]:
#include only approved/investigational/experimental
df_nddi = pd.merge(df_nddi, df_drug_status, left_on='Interaction_A(DrugBank_ID)', right_on='drug_name', how='left')
df_nddi = pd.merge(df_nddi, df_drug_status, left_on='Interaction_B(DrugBank_ID)', right_on='drug_name', how='left')

In [13]:
df_nddi.head()

,Interaction_A(DrugBank_ID),Interaction_B(DrugBank_ID),DrugBank ID_x,Name_x,CAS Number,Drug Type,KEGG Compound ID,KEGG Drug ID,PubChem Compound ID,PubChem Substance ID,...,BindingDB ID,TTD ID,DrugBank ID_y,Name_y,drug_name_x,include_x,inclusion_types_x,drug_name_y,include_y,inclusion_types_y
0,DB01183,DB06800,DB01183,Naloxone,465-65-6,SmallMoleculeDrug,C07252,D08249,5284596.0,46508816.0,...,54795.0,DAP000097,DB06800,Methylnaltrexone,NaN,NaN,NaN,DB06800,True,approved
1,DB01250,DB01109,DB01250,Olsalazine,15722-48-2,SmallMoleculeDrug,C07323,D00727,6003770.0,46506356.0,...,NaN,NaN,DB01109,Heparin,DB01250,True,approved,DB01109,True,"approved, investigational"
2,DB01586,DB01054,DB01586,Ursodeoxycholic acid,128-13-2,SmallMoleculeDrug,C07880,D00734,31401.0,46508795.0,...,53721.0,DNC000420,DB01054,Nitrendipine,DB01586,True,"approved, investigational",DB01054,True,"approved, investigational"
3,DB01212,DB01032,DB01212,Ceftriaxone,73384-59-5,SmallMoleculeDrug,C06683,D07659,5479530.0,46506458.0,...,50103601.0,DAP000145,DB01032,Probenecid,DB01212,True,approved,DB01032,True,"approved, investigational"
4,DB01326,DB01032,DB01326,Cefamandole,34444-01-4,SmallMoleculeDrug,C06879,D02344,456255.0,46508882.0,...,50350468.0,DAP000448,DB01032,Probenecid,NaN,NaN,NaN,DB01032,True,"approved, investigational"


In [14]:
df_nddi = df_nddi[['drug_name_x','Name_x','drug_name_y','Name_y']]

In [15]:
df_nddi = df_nddi.rename(columns={'drug_name_x': 'head_id', 'Name_x': 'head_name', 'drug_name_y': 'tail_id', 'Name_y': 'tail_name'})

In [16]:
df_nddi['head_type'] = 'drug'
df_nddi['head_ref'] = 'DrugBank'
df_nddi['tail_type'] = 'drug'
df_nddi['tail_ref'] = 'DrugBank'
df_nddi['predicate'] = 'nddi' 

In [17]:
df_nddi = df_nddi[['head_id', 'head_name', 'head_type','head_ref','tail_id', 'tail_name','tail_type','tail_ref','predicate']]

In [18]:
df_nddi.isna().sum()

head_id      3575
head_name      37
head_type       0
head_ref        0
tail_id      3332
tail_name      56
tail_type       0
tail_ref        0
predicate       0
dtype: int64

In [19]:
df_nddi.dropna(inplace=True)

In [20]:
df_nddi.isna().sum()

head_id      0
head_name    0
head_type    0
head_ref     0
tail_id      0
tail_name    0
tail_type    0
tail_ref     0
predicate    0
dtype: int64

In [21]:
df_nddi.drop_duplicates(inplace=True)

In [22]:
df_nddi.shape

(7408, 9)

In [23]:
#df_nddi.dtypes

In [ ]:
df_nddi.to_csv(r"...path_to\RDsqr-KG\KG_datasets\nddi.csv", index=False)